1)Написать приложение, которое собирает основные новости с сайтов news.mail.ru, lenta.ru, yandex.news
Для парсинга использовать xpath. Структура данных должна содержать:
название источника,
наименование новости,
ссылку на новость,
дата публикации

2)Сложить все новости в БД

In [1]:
import requests
from lxml import html
from pymongo import MongoClient
from pprint import pprint
from datetime import datetime, date

In [2]:
#задаем хедер для видимости использования браузера
header = {'User_Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
            AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}

#запускаем монго клиент с стандарными настройками
client = MongoClient('localhost', 27017)
#создаем новую бд 
db = client['News']
lenta = db.lenta
yandex = db.yandex
mail = db.mail


### lenta.ru

In [9]:
response = requests.get('https://lenta.ru/',
                            headers=header)
dom = html.fromstring(response.text)
data_lenta = []
items = dom.xpath("//section[contains(@class, 'top7')]//div[contains(@class, 'item')]")

for item in items:
    data = {}
    source = 'https://lenta.ru'
    name = item.xpath(".//h2//a//text | .//a//text()")
    date = item.xpath(".//h2//a//time//@datetime | .//a//time//@datetime")
    
    link = item.xpath(".//h2//a//@href | .//a//@href")
    if link[0][0] == '/':
        link = source + link[0]
    else:
        link = link[0]
        
    data['name'] = name[1].replace('\xa0', ' ')
    data['source'] = source
    data['link'] = link
    data['date'] = date[0]
    data_lenta.append(data)
    
pprint(data_lenta)

[{'date': ' 22:35,  6 июля 2020',
  'link': 'https://lenta.ru/news/2020/07/06/rus_us/',
  'name': 'Россия объявила об утрате доверия к США',
  'source': 'https://lenta.ru'},
 {'date': ' 23:52,  6 июля 2020',
  'link': 'https://lenta.ru/news/2020/07/06/sokolov/',
  'name': 'Стало известно о звонке историка Соколова в посольство Франции',
  'source': 'https://lenta.ru'},
 {'date': ' 23:36,  6 июля 2020',
  'link': 'https://lenta.ru/news/2020/07/06/opponenty/',
  'name': 'Адвокат Ефремова заявил о готовности снова лишиться '
          'профессионального статуса',
  'source': 'https://lenta.ru'},
 {'date': ' 23:19,  6 июля 2020',
  'link': 'https://lenta.ru/news/2020/07/06/us_uk/',
  'name': 'США оценили введенные Великобританией санкции против россиян',
  'source': 'https://lenta.ru'},
 {'date': ' 23:12,  6 июля 2020',
  'link': 'https://lenta.ru/news/2020/07/06/drug_house/',
  'name': 'Москвичи пожаловались на оккупировавших многоквартирный дом '
          'наркокурьеров',
  'source': 'h

### mail.ru

In [4]:
response = requests.get('https://news.mail.ru/',
                            headers=header)
dom = html.fromstring(response.text)
data_mail = []
items = dom.xpath("//div[@class='cols__inner']")

for i in items:
    if i.xpath(".//span[text()='Политика']"):
        break
        
items = i.xpath(".//li[@class='list__item'] | .//a[@class='newsitem__title link-holder']")

for item in items:
    data = {}
    name = item.xpath(".//span[@class='link__text']//text() | .//span[@class='newsitem__title-inner']//text()")
    link = item.xpath(".//a//@href | .//@href")
    
    m_response = requests.get(link[0], headers=header)
    m_dom = html.fromstring(m_response.text)
    
    date = m_dom.xpath("//span[@class='note__text breadcrumbs__text js-ago']//@datetime")
    date = date[0].replace('T', ' ')[:-9]
    
    sourse = m_dom.xpath("//a[@class='link color_gray breadcrumbs__link']//@href")[0]
    
    data['date'] = date
    data['sourse'] = sourse
    data['name'] = name[0]
    data['link'] = link[0]
    
    data_mail.append(data)
    
pprint(data_mail)

[{'date': '2020-07-06 18:07',
  'link': 'https://news.mail.ru/politics/42467935/',
  'name': 'Великобритания ввела санкции против россиян за нарушение прав '
          'человека',
  'sourse': 'https://rtvi.com'},
 {'date': '2020-07-06 18:03',
  'link': 'https://news.mail.ru/politics/42467751/',
  'name': 'Великобритания ввела персональные санкции против россиян по делу '
          'Магнитского',
  'sourse': 'http://www.tass.ru/'},
 {'date': '2020-07-06 17:42',
  'link': 'https://news.mail.ru/politics/42466192/',
  'name': 'Великобритания настроилась на санкции',
  'sourse': 'https://www.kommersant.ru/fm'},
 {'date': '2020-07-06 16:33',
  'link': 'https://news.mail.ru/politics/42466532/',
  'name': 'Главком украинских ВМС пообещал вернуть флот в Севастополь',
  'sourse': 'http://www.ria.ru'}]


### yandex

In [8]:
response = requests.get('https://yandex.ru/news/',
                            headers=header)
dom = html.fromstring(response.text)
data_ya = []
items = dom.xpath("//div[@aria-labelledby='personal_feed']//td[@class='stories-set__item']")

for item in items:
    data = {}
    main_link = 'https://yandex.ru'
    
    name = item.xpath(".//a//text()")[0]
    link = main_link + item.xpath(".//a//@href")[0]
    date = str(datetime.now().date()) + " " + item.xpath(".//div[@class='story__date']//text()")[0][-5:]  
    sourse = item.xpath(".//div[@class='story__date']//text()")[0][:-6]
    
    data['date'] = date
    data['sourse'] = sourse
    data['name'] = name
    data['link'] = link 
    
    data_ya.append(data)
    
pprint(data_ya)
    

[{'date': '2020-07-07 06:53',
  'link': 'https://yandex.ru/news/story/An-2_upal_okolo_nizhegorodskogo_Dzerzhinska--01de88b098c0dfb4ef39017dfe5f16f6?lr=80&lang=ru&stid=4Q4j2X0N42Gnzdps5BY5&persistent_id=104885698&rubric=personal_feed&from=index',
  'name': 'Ан-2 упал около нижегородского Дзержинска',
  'sourse': 'Интерфакс'},
 {'date': '2020-07-07 05:07',
  'link': 'https://yandex.ru/news/story/Glavkom_VMS_Ukrainy_zayavil_o_gotovnosti_k_boevym_dejstviyam_s_Rossiej--693bb1d6006f9d05a172669bfac44e75?lr=80&lang=ru&stid=QOVGapGAm87CZiUOwNpM&persistent_id=104748765&rubric=personal_feed&from=index',
  'name': 'Главком ВМС Украины заявил о готовности к боевым действиям с '
          'Россией',
  'sourse': 'Газета.Ru'},
 {'date': '2020-07-07 07:00',
  'link': 'https://yandex.ru/news/story/Boris_Korchevnikov_sfotografirovalsya_na_vypiske_iz_roddoma--206fc0bdd502f91531f62529612d9fd1?lr=80&lang=ru&stid=KeLOocAqM_C8c4UytTsy&persistent_id=104745535&rubric=personal_feed&from=index',
  'name': 'Борис 

In [6]:
yandex.insert_many(data_ya)
mail.insert_many(data_mail)
lenta.insert_many(data_lenta)

In [14]:
date

[' 19:52,  6 июля 2020']

In [20]:
datetime.strptime(date[0].replace(',',''),'%Y-%m-%d')

ValueError: time data ' 19:52  6 июля 2020' does not match format '%Y-%m-%d'

In [13]:
date[0].replace(',','')

' 19:52  6 июля 2020'